### Schema

0. x coordinate
1. y coordinate
2. timestamp
3. pen-up
4. azimuth angle
5. altitude angle
6. Pressure

In [11]:
import pandas as pd
import numpy as np
from os import path
import sys


#plotting
from matplotlib import pyplot as plt
%matplotlib inline

#processing
from scipy.signal import resample


In [3]:
#ml
from tensorflow.keras.callbacks import EarlyStopping
import mlflow
import mlflow.tensorflow
tf.__version__

'2.9.1'

In [5]:
pip install hyperopt

     |████████████████████████████████| 1.6 MB 72 kB/s eta 0:00:012
     |████████████████████████████████| 78 kB 137 kB/s eta 0:00:01
     |████████████████████████████████| 829 kB 53 kB/s eta 0:00:011
     |████████████████████████████████| 2.0 MB 28 kB/s eta 0:00:014
     |████████████████████████████████| 200 kB 41 kB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491058 sha256=37f0b531d2482dc1809fa88cf1b6638ef4043e1f90bdd90cb1a1eef22dbe1234
  Stored in directory: /root/.cache/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
Successfully built future
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [21]:
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials
from hyperopt.pyll.base import scope 

In [22]:
tpe.suggest

<function hyperopt.tpe.suggest(new_ids, domain, trials, seed, prior_weight=1.0, n_startup_jobs=20, n_EI_candidates=24, gamma=0.25, verbose=True)>

In [7]:
features=['x','y','pen_up','pressure']

Asegurar que la muestra y labels son las correctas

In [8]:
doc_path = "/data/elekin/doc"
metadf=pd.read_csv(path.join(doc_path,"metadata-202106-v1.csv"),index_col=0)
metadf[metadf["temblor"]=='si']

,abs_path,relative_path,temblor,Sexo,TRS,age,level
name,,,,,,,
T001,/data/elekin/data/origin/ethw/protocolo_temblo...,T001/session00001/u00005s00001_hw00001.svc,si,M,36,74,2
T002_1,/data/elekin/data/origin/ethw/protocolo_temblo...,T002/session00001/u00006s00001_hw00002.svc,si,M,40,71,2
T002_2,/data/elekin/data/origin/ethw/protocolo_temblo...,T002/session00001/u00006s00001_hw00001.svc,si,M,40,71,2
T005,/data/elekin/data/origin/ethw/protocolo_temblo...,T005/session00001/u00009s00001_hw00001.svc,si,M,29,69,2
T006,/data/elekin/data/origin/ethw/protocolo_temblo...,T006/session00001/u00010s00001_hw00001.svc,si,F,19,77,2
T008,/data/elekin/data/origin/ethw/protocolo_temblo...,T008/session00001/u00013s00001_hw00001.svc,si,F,3,68,1
T009,/data/elekin/data/origin/ethw/protocolo_temblo...,T009/session00001/u00014s00001_hw00001.svc,si,F,16,66,2
T010,/data/elekin/data/origin/ethw/protocolo_temblo...,T010/session00001/u00015s00001_hw00002.svc,si,M,16,61,2
T012_1,/data/elekin/data/origin/ethw/protocolo_temblo...,T012/session00001/u00005s00001_hw00001.svc,si,F,11,51,1


In [9]:
# Creating a data structure with subjects time series and tremor levels
 
X = []
y = []

# shape input to be 3D [samples, timesteps, features]
for path, level in zip(metadf.abs_path, metadf.level):
    df = pd.read_csv(path, sep="\s+",header=None,names=features,skiprows=1,usecols=[0,1,3,6])
    X.append(resample(df.values.astype('int16'), 4096))
    y.append(level)
dim = np.array(X).shape
print("{} samples,{} timesteps, {} features".format(dim[0], dim[1], dim[2]))

53 samples,4096 timesteps, 4 features


In [12]:
idx = 0
label = y[idx]
plt.scatter(X[idx][:,0], X[idx][:,1], c=X[idx][:,3], cmap='jet')
_=plt.title("Subjet {}, tremor severity group: {}".format(metadf.index[idx], label))

In [14]:
seed = 38
n_outputs = 3
mini_batch_size=4

n_timesteps = np.array(X).shape[1]
n_features = np.array(X).shape[2]

data_size = np.array(X).shape[0]
train_split = 0.67
test_split = 0.33
train_size = int(train_split * data_size)
test_size = int(test_split * data_size)

shuffle_buffer = data_size
steps_per_epoch = round(data_size/mini_batch_size)


AUTOTUNE = tf.data.experimental.AUTOTUNE # Adapt preprocessing and prefetching dynamically to reduce GPU and CPU idle time

MLFLOW_TRACKING_URI = "http://mlflow_server:5001"

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment('/archimedes-dl')

In [15]:
# label encoding
labels = tf.one_hot(y,n_outputs)

In [16]:
# train, val split
dataset = tf.data.Dataset.from_tensor_slices((X, labels))
full_dataset = dataset.shuffle(shuffle_buffer, seed=seed)
train_dataset = full_dataset.take(train_size).batch(mini_batch_size).prefetch(AUTOTUNE).cache()
test_dataset = full_dataset.skip(train_size).batch(mini_batch_size).prefetch(AUTOTUNE).cache()

In [11]:
# for feat, targ in train_dataset.shuffle(shuffle_buffer):
# #         print ('Features: {0}'.format(feat, targ))
#     print ('Target train: {1}'.format(feat, targ))
    
# for feat, targ in test_dataset.shuffle(shuffle_buffer):
# #         print ('Features test: {}, Target test: {}'.format(feat, targ))
#         print ('Target test: {1}'.format(feat, targ))

# print("{0} train batches and {1} test batches of {2} mini batch size and {3} steps per epoch".format(len(train_dataset), 
#                                                                               len(test_dataset),
#                                                                               mini_batch_size,
#                                                                                 steps_per_epoch))

In [17]:
## search space

space = {'rate'       : hp.uniform('rate',0.01,0.5),
         'units'      : hp.quniform('units',32,64,128),
         'layers'     : hp.quniform('layers',1,6,1),
        }

In [18]:
def f_nn(hparams):
    
    model = tf.keras.models.Sequential()
    if hparams['layers'] == 1:
        model.add(tf.keras.layers.LSTM(hparams['units'], activation=tf.nn.tanh, return_sequences=False, input_shape=(n_timesteps,n_features)))
        model.add(tf.keras.layers.Dropout(hparams['rate']))
    else:
        model.add(tf.keras.layers.LSTM(hparams['units'], activation=tf.nn.tanh, return_sequences=True, input_shape=(n_timesteps,n_features)))
        model.add(tf.keras.layers.Dropout(hparams['rate']))
        
        for i in range(hparams['layers']-2):
            model.add(tf.keras.layers.LSTM(hparams['units'], activation=tf.nn.tanh, return_sequences=True ))
            model.add(tf.keras.layers.Dropout(hparams['rate']))
    
        model.add(tf.keras.layers.LSTM(hparams['units'], activation=tf.nn.tanh, return_sequences=False ))
        model.add(tf.keras.layers.Dropout(hparams['rate']))
    
    model.add(tf.keras.layers.Dense(3, activation=tf.nn.sigmoid))
              
    tf.keras.backend.clear_session()# avoid clutter from old models and layers, especially when memory is limited
    tf.random.set_seed(seed) # establecemos la semilla para tensorflow
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
              
    result = model.fit(train_dataset, 
                       use_multiprocessing=True, 
                       validation_data=test_dataset, 
                       batch_size=4, epochs=200)
    
    validation_acc = np.amax(result.history['val_accuracy'])
    print('Best validation loss of epoch:', validation_acc)
    return {'accuracy': validation_acc, 
            'status': STATUS_OK, 
            'model': model, 
            'params': hparams}

In [24]:
run = 'lstm/hp'
trials = Trials()
best = fmin(f_nn, 
            space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials)

TPE is being used as the default algorithm.


  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

job exception: Dimension value must be integer or None or have an __index__ method, got value '0.0' with type '<class 'float'>'



  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]


TypeError: Dimension value must be integer or None or have an __index__ method, got value '0.0' with type '<class 'float'>'

In [29]:
trials

In [25]:
mlflow.end_run()